In [23]:
import os
import torch
import torchvision.transforms as transforms
from google.colab import drive
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drive.mount('/content/drive')
validation_dataset_path = "/content/drive/MyDrive/test_images" #test dataset for analysis
model_path = "/content/drive/MyDrive/efficient_net_models/model_5epochs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
model = torch.load(model_path)
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [25]:
#transformation class to create object of
class ImageTransformer:
    def __init__(self):
        self._image_size = (224, 224)

        self.transform = transforms.Compose([
            transforms.Resize(self._image_size),
            transforms.ToTensor(),
            # transforms.Normalize(mean=self._mean, std=self._std)
        ])

In [26]:
#transforms image into what required as input for model
image_processor = ImageTransformer()


In [39]:
id_to_label = {0: 'Packing_List', 1: 'Bill_of_Lading', 2: 'Certificate_of_Origin',
                  3: 'Insurance_Certificate', 4: 'Covering_Schedule'}

In [40]:
import pandas as pd
a = pd.DataFrame
data ={"Actual_Class": [], "Predicted_Class": []}
from PIL import Image
for file in os.listdir(validation_dataset_path):
  img = Image.open(os.path.join(validation_dataset_path,file)).convert("RGB")
  #print(file[:-13])
  transformed_input_image = image_processor.transform(img)
  transformed_input_image = transformed_input_image.reshape(1,3,224,224)
  #print(transformed_input_image.shape)
  output = model(transformed_input_image)
  _, predicted_idx = torch.max(output, 1)
  #print(output.shape)
  data["Actual_Class"].append(file[:-13])
  data["Predicted_Class"].append(id_to_label[int(predicted_idx)])
  #print(id_to_label[int(predicted_idx)])
a = pd.DataFrame(data)

In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score
classes = a['Actual_Class'].unique()
precisions = []
recalls = []
f1_scores = []

for cls in classes:
    actual = a['Actual_Class'].map(lambda x: 1 if x == cls else 0)
    predicted = a['Predicted_Class'].map(lambda x: 1 if x == cls else 0)

    precision = precision_score(actual, predicted)
    recall = recall_score(actual, predicted)
    f1 = f1_score(actual, predicted)

    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
metrics_df = pd.DataFrame({
    'Class': classes,
    'Precision': precisions,
    'Recall': recalls,
    'F1-Score': f1_scores
})

print(metrics_df)

                   Class  Precision  Recall  F1-Score
0           Packing_List        1.0     1.0       1.0
1  Certificate_Of_Origin        0.0     0.0       0.0
2         Bill_of_lading        0.0     0.0       0.0
3  Insurance_Certificate        1.0     1.0       1.0
4      Covering_Schedule        1.0     1.0       1.0


In [50]:
metrics_df.to_csv("metric_df.csv")